In [103]:
# !pip install pandas_ml
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from keras.layers import Dense, Input
from keras.layers.merge import concatenate
from keras.models import model_from_json
%matplotlib inline
import tensorflow as tf
tf.test.gpu_device_name()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


''

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [120]:
json_file = open('dress_or_top_processed.json','r')
loaded_model_json = json_file.read()
json_file.close()
dressOrTopModel = model_from_json(loaded_model_json)
dressOrTopModel.load_weights("dress_or_top_processed.h5")
print("Loaded model from disk")

json_file = open('dress_model_v1.json','r')
loaded_model_json = json_file.read()
json_file.close()
dressesModel = model_from_json(loaded_model_json)
dressesModel.load_weights('dress_model_v1.h5')
print("Loaded model from disk")

json_file = open('processed_tops_model_v1.json','r')
loaded_model_json = json_file.read()
json_file.close()
topsModel = model_from_json(loaded_model_json)
topsModel.load_weights('processed_tops_model_v1.h5')
print("Loaded model from disk")

Loaded model from disk
Loaded model from disk
Loaded model from disk


In [0]:
dressOrTopModel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
dressesModel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
topsModel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
####Model building

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils


from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline


In [0]:
fashion_train = pd.read_csv("fashion_processed.csv")
dressOrTop = []

for index,row in fashion_train.iterrows():
  if row.Category in range(18,25):
    dressOrTop.append(0)
#     print(row['DressOrTop'])
  elif row.Category in range(25,31):
    dressOrTop.append(1)
  else:
    dressOrTop.append(2)
    
fashion_train['DressOrTop'] = dressOrTop

In [0]:

test = pd.read_csv('test.csv')
fashion_test = test.iloc[76545:131985].reset_index()

In [125]:
training = fashion_train

percentage_split = 0.8
max_words = 1000
batch_size = 1000
epochs = 80

train = training.sample(frac=1).reset_index(drop=True) #Shuffles the rows in the Train.CSV because the rows are arranged neatly in the order of big categories (Beu, Fas, mob)
train_size = int(len(train) * percentage_split) 
train_title = train['title'][:train_size] #Takes 70% of the training data to train
train_Category = train['DressOrTop'][:train_size] #Takes 70% of the training data to train

eval_title = train['title'][train_size:] #Takes remaining 30% of the training data to test
eval_Category = train['DressOrTop'][train_size:] #Takes remaining 30% of the training data to test

train_title = train['title']
train_Category = train['DressOrTop']

tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_title) # only fit on train

x_train = tokenize.texts_to_matrix(train_title)
x_eval = tokenize.texts_to_matrix(eval_title)

encoder = LabelEncoder()
encoder.fit(train_Category)
y_train = train_Category
y_eval = eval_Category
y_train = encoder.transform(train_Category)
y_eval = encoder.transform(eval_Category)


num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_eval = utils.to_categorical(y_eval, num_classes)


print('Found %d unique words.' % len(tokenize.word_index))
# print(train_title)

Found 39707 unique words.


In [126]:
### Split dress and top


test_title = fashion_test['title']
x_test = tokenize.texts_to_matrix(test_title)
test_predict = dressOrTopModel.predict(x_test, batch_size=batch_size, verbose=1)
test_predict = pd.DataFrame(data=test_predict)
test_predict = test_predict.idxmax(axis=1)
test_predict

55440/55440 [==============================] - 4s 74us/step


0        0
1        0
2        0
3        0
4        1
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
55410    1
55411    0
55412    1
55413    1
55414    1
55415    1
55416    1
55417    1
55418    1
55419    1
55420    1
55421    1
55422    1
55423    1
55424    1
55425    1
55426    2
55427    0
55428    1
55429    1
55430    0
55431    1
55432    1
55433    1
55434    1
55435    1
55436    1
55437    1
55438    0
55439    1
Length: 55440, dtype: int64

In [0]:
#Download prediction classes given test set
test_predict.to_csv('dress_or_top_predict.csv') 
from google.colab import files
files.download('dress_or_top_predict.csv')

In [113]:
# pred = pd.read_csv('dress_or_top_predict.csv')
# pred = pred['0.1']
itemIDs = fashion_test['itemid']
first_pred = pd.DataFrame()
first_pred['itemid'] = itemIDs
first_pred['DressOrTop'] = test_predict
first_pred['title'] = test_title
first_pred

,itemid,DressOrTop,title
0,670968580,0,women casual o neck long sleeve lace patchwork...
1,475599610,0,zaful long dress vintage wanita v neck lengan ...
2,1652671018,0,dress skater swing o neck tanpa lengan desain ...
3,1705634904,0,ok dress slim sexy deep v neck lengan panjang ...
4,1789421087,1,noa korean lace embroidery womens crew neck lo...
5,1818944472,0,vintage women christmas lace patchwork o neck ...
6,1341150340,0,ready stock dress v neck tanpa lengan desain p...
7,1021592916,0,women patchwork long sleeve lace o neck beach ...
8,1252778162,0,fashion wanita dress bodycon bandage pensil la...
9,1309386587,0,fashion bohemian women lady dress chiffon chro...


In [0]:
dresses = first_pred.loc[first_pred['DressOrTop']==0].reset_index()
tops = first_pred.loc[first_pred['DressOrTop']==1].reset_index()
others = first_pred.loc[first_pred['DressOrTop']==2].reset_index()

In [115]:
dresses_title = dresses['title']
x_test = tokenize.texts_to_matrix(dresses_title)
dress_predict = dressesModel.predict(x_test, batch_size=batch_size, verbose=1)
dress_predict = pd.DataFrame(data=dress_predict)
dress_predict = dress_predict.idxmax(axis=1)

dresses['Cat2'] = dress_predict
dresses['Cat2'] = [label + 18 for label in dress_predict]
dresses

6819/6819 [==============================] - 1s 159us/step


,index,itemid,DressOrTop,title,Cat2
0,0,670968580,0,women casual o neck long sleeve lace patchwork...,18
1,1,475599610,0,zaful long dress vintage wanita v neck lengan ...,22
2,2,1652671018,0,dress skater swing o neck tanpa lengan desain ...,22
3,3,1705634904,0,ok dress slim sexy deep v neck lengan panjang ...,22
4,5,1818944472,0,vintage women christmas lace patchwork o neck ...,22
5,6,1341150340,0,ready stock dress v neck tanpa lengan desain p...,18
6,7,1021592916,0,women patchwork long sleeve lace o neck beach ...,18
7,8,1252778162,0,fashion wanita dress bodycon bandage pensil la...,20
8,9,1309386587,0,fashion bohemian women lady dress chiffon chro...,18
9,10,1770261713,0,gaun maxi wanita tanpa lengan dengan bahan sif...,18


In [116]:
tops_title = tops['title']
x_test = tokenize.texts_to_matrix(tops_title)
tops_predict = topsModel.predict(x_test, batch_size=batch_size, verbose=1)
tops_predict = pd.DataFrame(data=tops_predict)
tops_predict = tops_predict.idxmax(axis=1)

tops['Cat2'] = tops_predict
tops['Cat2'] = [label + 25 for label in tops_predict]
tops

48525/48525 [==============================] - 4s 73us/step


,index,itemid,DressOrTop,title,Cat2
0,4,1789421087,1,noa korean lace embroidery womens crew neck lo...,26
1,31,1440613377,1,bayar di tempat aus kaos t shirt wanita sexy l...,25
2,37,1159498284,1,romper v neck lengan pendek sexy bahan sifon m...,26
3,41,1748151612,1,men and women can wear spring summer korean ve...,26
4,57,1169512512,1,blus v neck lengan panjang lotus bahan sifon w...,27
5,75,1710700151,1,shein sexy wanita dua piece set elegan navy fl...,25
6,97,1560352999,1,fl blus sifon wanita v neck lengan panjang war...,27
7,126,1247454798,1,sq romper wanita sexy tanpa lengan model v nec...,25
8,158,1240838006,1,women fahsion sexy o neck sleeveless dot ptint...,25
9,175,477567828,1,romper wanita o neck tanpa lengan backless bah...,29


In [0]:
others['Cat2'] = 17

In [118]:
predictions = dresses.append([tops,others],ignore_index=True)
predictions

,index,itemid,DressOrTop,title,Cat2
0,0,670968580,0,women casual o neck long sleeve lace patchwork...,18
1,1,475599610,0,zaful long dress vintage wanita v neck lengan ...,22
2,2,1652671018,0,dress skater swing o neck tanpa lengan desain ...,22
3,3,1705634904,0,ok dress slim sexy deep v neck lengan panjang ...,22
4,5,1818944472,0,vintage women christmas lace patchwork o neck ...,22
5,6,1341150340,0,ready stock dress v neck tanpa lengan desain p...,18
6,7,1021592916,0,women patchwork long sleeve lace o neck beach ...,18
7,8,1252778162,0,fashion wanita dress bodycon bandage pensil la...,20
8,9,1309386587,0,fashion bohemian women lady dress chiffon chro...,18
9,10,1770261713,0,gaun maxi wanita tanpa lengan dengan bahan sif...,18


In [0]:
predictions = pd.merge(fashion_test, predictions, on="itemid")


In [0]:
predictions.to_csv("fashion_split_predictions.csv")
files.download("fashion_split_predictions.csv")